## 1. Result Description

For the same object, the robot will eventually generate 8 csv files from different angles and different distances, because we have 6 objects to be detected so we did $6 \times 8 = 48 $ experiments, the total number of files is 48 as well.

The filename of the result file identifies the detected object and the coordinates of the position where the robot detected it. In the file, it contains the results the model detects the target during the rotation. The detection result includes the category of the object to be detected, and the probability of predicting the category, also known as the confidence.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('experiment_results_ssd/person/results_performance_person_0_m4.csv', index_col='Unnamed: 0')

In [3]:
df

,category,score
0,none,0.0
1,none,0.0
2,none,0.0
3,none,0.0
4,none,0.0
...,...,...
428,none,0.0
429,none,0.0
430,none,0.0
431,none,0.0


## 2. Model speed

In this experiment, as all the nodes are same. The only major element that affects the detection speed is model. As we know, for an object, the robit rotates same angle at each position so the time of rotation is same. Thus, we will explore the model speed by comparing how many data streams each pre-trained model produced in a fixed time.

In [14]:
# Define a function to count the number of results
def result_counter(results, model_type):

    objects_name = ['car', 'bus', 'person', 'chair', 'suitcase', 'monitor']
    positions = ['m6_0', 'm4_0', '4_0', '6_0', '0_6', '0_4', '0_m4', '0_m6']
    new_df = pd.DataFrame(columns=objects_name, index=positions)

    for result in results:
        obj, position = get_obj_pos(result)

        df = pd.read_csv(result, index_col='Unnamed: 0')
        num_row = df.shape[0]
        new_df.at[position, obj] = num_row

    new_df.to_csv(os.path.join('./process_result', model_type + '_result_count.csv'))

In [15]:
ssd_result_names = get_file_names('./experiment_results_ssd')
yolo_result_names = get_file_names('./experiment_results_yolov3')

# Execute the function for ssd and yolov3
result_counter(ssd_result_names, 'ssd')
result_counter(yolo_result_names, 'yolov3')

In [16]:
df_ssd = pd.read_csv('./process_result/ssd_result_count.csv', index_col='Unnamed: 0')
df_ssd

,car,bus,person,chair,suitcase,monitor
m6_0,415,439,430,442,424,426
m4_0,380,434,417,406,415,417
4_0,374,460,424,425,418,426
6_0,368,402,452,446,422,457
0_6,385,435,412,421,400,433
0_4,366,441,423,440,404,426
0_m4,372,404,433,443,443,442
0_m6,374,420,413,409,449,431


In [17]:
df_yolov3 = pd.read_csv('./process_result/yolov3_result_count.csv', index_col='Unnamed: 0')
df_yolov3

,car,bus,person,chair,suitcase,monitor
m6_0,188,174,187,179,190,186
m4_0,173,182,183,178,192,179
4_0,173,212,174,181,183,177
6_0,170,169,182,182,178,166
0_6,167,177,178,202,172,184
0_4,165,167,183,188,182,179
0_m4,168,161,187,183,183,174
0_m6,166,180,181,181,178,177


In [18]:
avg_counts_ssd = df_ssd.sum().sum() / 48
avg_counts_yolov3 = df_yolov3.sum().sum() / 48
print(f"The Average number of detection results in SSD is: {int(avg_counts_ssd)}.\nThe Average number of detection results in Yolov3 is: {int(avg_counts_yolov3)}.")

The Average number of detection results in SSD is: 419.
The Average number of detection results in Yolov3 is: 179.


We can observe from the above tables that the average result number in an experiment produced by the SSD model is 419. The average result number in an experiment produced by the Yolov3 model is 179. In a fixed period, the SSD model is $\frac{179}{429} \times 100\% = 42.7\%$ faster than Yolov3 model.

## 3. Data Processing

When the robot is at the initial position, the direction is 45 degrees offset to the object.
The object is not in the capture range of the camera. E.g. In figure 1, we can see there is no object displayed in the GUI. When the control node is launched, during the process of robot rotation, the object enters the range of the camera, as it is shown in figure 2. Thus, between the period that the initial direction and the direction the object can enter into the camera range,  The rotation of the robot starts at the same time as the start of the save node to receive data, so the detection result is invalid from the start time to the time when the object enters the range that the camera can capture. Similarly, in the process of completing the 90-degree rotation, the detection result of the time when the object is about to leave the camera range and the time when the robot completes the 90-degree rotation is also invalid. In the project code, we define that the return value of the detection result of this class is None, and the score is 0. Therefore, we need to remove this part of invalid data. In addition, when detecting some objects, the pre-trained model may not be able to detect the object in the whole process, so all the return values of the detection results are invalid. Therefore, in order not to eliminate the results sent because the model itself did not detect the object , we only filter the detection results with valid data greater than 50% of the total data. Figure 3 shows which part of invalid data we elimited.

In [7]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np

In [8]:
import time

round(time.time(), 2)

1651868636.52

In [9]:
# Get all the result files name
def get_file_names(path):
    # param: path - there are two categories results, one is detected by ssd, the other one is Yolov3.
    # so we have to indicate which kind of results we are going to process
    file_names = [os.path.join(root, name) for root, _, files in os.walk(path) for name in files if name.endswith('csv')]
    return file_names

ssd_result_names = get_file_names('./experiment_results_ssd')
yolo_result_names = get_file_names('./experiment_results_yolov3')

print(f"There are {len(ssd_result_names)} SSD results")
print(f"There are {len(yolo_result_names)} Yolov3 results")

There are 48 SSD results
There are 48 Yolov3 results


In [10]:
ssd_result_names

['./experiment_results_ssd/chair/results_performance_chair_m6_0.csv',
 './experiment_results_ssd/chair/results_performance_chair_0_m4.csv',
 './experiment_results_ssd/chair/results_performance_chair_0_m6.csv',
 './experiment_results_ssd/chair/results_performance_chair_4_0.csv',
 './experiment_results_ssd/chair/results_performance_chair_0_4.csv',
 './experiment_results_ssd/chair/results_performance_chair_6_0.csv',
 './experiment_results_ssd/chair/results_performance_chair_m4_0.csv',
 './experiment_results_ssd/chair/results_performance_chair_0_6.csv',
 './experiment_results_ssd/car/results_performance_car_0_m4.csv',
 './experiment_results_ssd/car/results_performance_car_0_4.csv',
 './experiment_results_ssd/car/results_performance_car_0_m6.csv',
 './experiment_results_ssd/car/results_performance_car_m4_0.csv',
 './experiment_results_ssd/car/results_performance_car_4_0.csv',
 './experiment_results_ssd/car/results_performance_car_6_0.csv',
 './experiment_results_ssd/car/results_performance_

In [19]:
# Define a function to get the object name, position according to the result file path
def get_obj_pos(result_path):
    split_file_name = result_path.split('/')[-1].split('_')
    obj = split_file_name[2]
    position = split_file_name[3] + "_" + split_file_name[-1].replace('.csv', '')
    return obj, position

# Define a function to eliminate the none value in the file, and save them as a new file.
def eliminate_none(results_list, model_type):
    # eliminate every result file
    for result in results_list:
        object_name, position = get_obj_pos(result)
        df = pd.read_csv(result, index_col="Unnamed: 0")    # open the result file
        # df = df.drop(columns=['Unnamed: 0'])
        for index, row in df.iterrows():
            if row['category'] == 'none':
                df = df.drop([index])
        df.to_csv(os.path.join('./cleaned_data', model_type, object_name + "_" + position + ".csv"), encoding='utf-8')

In [13]:
# eliminate the none for ssd results
eliminate_none(ssd_result_names, "ssd")
# eliminate the none of yolov3 results
eliminate_none(yolo_result_names, 'yolov3')

## 4. Sensitivity

The detection rate is used to measure the proportion of objects in the environment detected by the pre-trained model in all experiments. During the detection process, the model may not be able to identify the object according to the image captured by the camera. At this time, in the result published by the detection node, the category is none and the confidence is 0. We regard this result as failed. In a single experiment, if the proportion of failed results reache more than 50%, we determine that the robot cannot detect the object at this position. By analyzing the data, we can get the following table:

In [20]:
import pandas as pd

In [21]:
# Get the cleaned files_paths of ssd
clean_ssd_results = get_file_names('./cleaned_data/ssd')

# Get the cleaned files_paths of yolov3
clean_yolov3_results = get_file_names('./cleaned_data/yolov3')

In [22]:
# Define a function that can return a new dataframe with the columns are the objects, the indexes are the positions
def create_new_df():
    objects_name = ['car', 'bus', 'person', 'chair', 'suitcase', 'monitor']
    positions = ['m6_0', 'm4_0', '4_0', '6_0', '0_6', '0_4', '0_m4', '0_m6']
    new_df = pd.DataFrame(columns=objects_name, index=positions)

    return new_df

In [23]:
# Define a function to evaluate whether the detection is success or failed
def detect_evl(result_paths, model_type):
    # Define the objects name and positions as the rows and index of table
    objects_name = ['car', 'bus', 'person', 'chair', 'suitcase', 'monitor']
    positions = ['m6_0', 'm4_0', '4_0', '6_0', '0_6', '0_4', '0_m4', '0_m6']
    empty_df = pd.DataFrame(index=positions, columns=objects_name)
    for result in result_paths:
        df = pd.read_csv(result)
        split_result = result.split('/')[-1].split('_')
        obj = split_result[0]
        pos = split_result[1] + '_' + split_result[-1].replace('.csv', '')

        # If the number of rows is less than 10, we count it as a failed detection, so we fill the corresponding cell with value False
        # If the number of rows is greater than 10, we count it as a success detection, so we fill the corresponding cell with value True
        if df.shape[0] < 10:
            empty_df.at[pos, obj] = False
        else:
            empty_df.at[pos, obj] = True
    empty_df.to_csv(os.path.join('./process_result', model_type + '_detection_rate.csv'), encoding='utf-8')

In [24]:
# detect for ssd
detect_evl(clean_ssd_results, "ssd")

# detect for yolov3
detect_evl(clean_yolov3_results, "yolov3")

In [25]:
# Open the detection status of ssd
df_ssd = pd.read_csv('./process_result/ssd_detection_rate.csv', index_col='Unnamed: 0')
df_ssd

,car,bus,person,chair,suitcase,monitor
m6_0,True,True,True,False,True,False
m4_0,True,True,True,True,True,False
4_0,True,True,True,True,True,False
6_0,True,True,True,False,True,False
0_6,True,True,True,True,True,True
0_4,True,True,True,True,True,True
0_m4,True,True,True,True,True,True
0_m6,True,True,True,True,True,True


In [39]:
# Open the detection status of yolov3
df_yolov3 = pd.read_csv('./process_result/yolov3_detection_rate.csv', index_col='Unnamed: 0')
df_yolov3

,car,bus,person,chair,suitcase,monitor
m6_0,True,True,True,False,True,True
m4_0,True,True,True,True,False,False
4_0,True,True,True,True,False,False
6_0,True,True,True,True,False,True
0_6,True,False,True,True,False,False
0_4,True,True,True,True,False,False
0_m4,True,True,True,True,True,False
0_m6,True,True,True,True,True,False


We can observe from above tables that the success detection of SSD model is 42 out of 48 detections, the success detection of Yolov3 model is 13 out of 48 detections.
The success rate of SSD and Yolov3 in the detection is:

$DetectRate_{ssd} = \frac{42}{48} \times 100\% = 87.5\% $ \
$DetectRate_{yolov3} = \frac{35}{48} \times 100\% = 73.0\% $


## 5. Accuracy

In section 3, we evaluated when the camera capture the image in the environment, whether the model can percept there is an object. In this section, we are going to evaluate the accuracy of detection, which means how many correct detections the model made in the experiment. We will evaluate from two aspects, in the below tables, we can see the accuracy in each detection.

In [45]:
def accuracy_evl(cleaned_results, model_type):
    # Create a new dataframe to store the processed result
    new_df = create_new_df()
    # iterate the detection results
    for result in cleaned_results:
        # Get the object name and robot position
        obj = result.split('/')[-1].split('_')[0]
        position = result.split('/')[-1].split('_')[1] + '_' + result.split('/')[-1].split('_')[-1].replace('.csv', '')

        # Open the detection result
        df = pd.read_csv(result, index_col='Unnamed: 0')
        # num of results
        num_det = df.shape[0]
        # calculate how many correct detection
        detections = df['category'].value_counts()
        if obj not in detections.index:
            new_df.at[position, obj] = 0.0
            continue
        correct_rate = detections[obj] / num_det
        new_df.at[position, obj] = correct_rate
    new_df.loc['avg_acc'] = new_df.sum() / 8
    new_df.to_csv(os.path.join('./process_result', model_type + "_detection_accuracy.csv"))

In [46]:
clean_ssd_results = get_file_names('./cleaned_data/ssd')
clean_yolov3_results = get_file_names('./cleaned_data/yolov3')

accuracy_evl(clean_ssd_results, 'ssd')
accuracy_evl(clean_yolov3_results, 'yolov3')

ssd_acc_df = pd.read_csv('./process_result/ssd_detection_accuracy.csv', index_col=['Unnamed: 0'])
yolov3_acc_df = pd.read_csv('./process_result/yolov3_detection_accuracy.csv', index_col=['Unnamed: 0'])

In [47]:
ssd_acc_df

,car,bus,person,chair,suitcase,monitor
m6_0,0.537162,0.418605,1.000000,0.000000,0.000000,0.0
m4_0,0.005988,0.568116,0.996241,1.000000,0.000000,0.0
4_0,0.401384,0.377581,1.000000,0.769231,0.029412,0.0
6_0,0.032864,0.548387,0.992908,0.000000,0.000000,0.0
0_6,0.009615,0.000000,1.000000,0.000000,0.000000,0.0
0_4,0.049180,0.000000,0.989091,0.000000,0.000000,0.0
0_m4,0.000000,0.000000,0.971429,0.000000,0.500000,0.0
0_m6,0.004545,0.000000,1.000000,0.000000,0.000000,0.0
avg_acc,0.130092,0.239086,0.993708,0.221154,0.066176,0.0


In [48]:
yolov3_acc_df

,car,bus,person,chair,suitcase,monitor
m6_0,0.987097,0.868687,1.0,0.800000,1.0,0.0
m4_0,1.000000,0.892655,1.0,0.983871,0.0,0.0
4_0,1.000000,0.649215,1.0,0.891667,0.0,0.0
6_0,1.000000,0.472973,1.0,0.990991,0.0,0.0
0_6,1.000000,0.000000,1.0,0.976744,1.0,0.0
0_4,0.680000,0.000000,1.0,0.824742,0.0,0.0
0_m4,1.000000,0.000000,1.0,0.921348,1.0,0.0
0_m6,1.000000,0.000000,1.0,1.000000,1.0,0.0
avg_acc,0.958387,0.360441,1.0,0.923670,0.5,0.0


From the tables we can tell the both model could not detect monitor very well. Both models have good accuracy for the detection on person. The yolov3 model has better performance than SSD as its average accuracy on other objects are higher than SSD. For further exploration, we will check the average confidence in each experiment.

In [74]:
def confidence_evl(cleaned_results, model_type):
    # Create a new dataframe to store the processed result
    new_df = create_new_df()
    # iterate the detection results
    for result in cleaned_results:
        # Get the object name and robot position
        obj = result.split('/')[-1].split('_')[0]
        position = result.split('/')[-1].split('_')[1] + '_' + result.split('/')[-1].split('_')[-1].replace('.csv', '')

        # Open the detection result
        df = pd.read_csv(result, index_col='Unnamed: 0')
        # num of results
        num_det = df.shape[0]
        # calculate how many correct detection
        detections = df['category'].value_counts()
        if obj not in detections.index:
            new_df.at[position, obj] = 0.0
            continue
        # correct_rate = detections[obj] / num_det
        # new_df.at[position, obj] = correct_rate
        confidences = []
        for index, row in df.iterrows():
            if row['category'] == obj:
                confidences.append(row['score'])
        avg_confidence = np.array(confidences).sum() / detections[obj]
        new_df.at[position, obj] = avg_confidence
        # new_df.loc['avg_confi'] = new_df.sum() / 8
    # new_df.loc['avg_acc'] = new_df.sum() / 8
    new_df.to_csv(os.path.join('./process_result', model_type + "_confidence.csv"))

In [75]:
confidence_evl(clean_ssd_results, 'ssd')
confidence_evl(clean_yolov3_results, 'yolov3')

In [76]:
ssd_confi = pd.read_csv('./process_result/ssd_confidence.csv', index_col='Unnamed: 0')
yolov3_confi = pd.read_csv('./process_result/yolov3_confidence.csv', index_col='Unnamed: 0')

In [77]:
ssd_confi

,car,bus,person,chair,suitcase,monitor
m6_0,0.659119,0.549630,0.640077,0.000000,0.000,0.0
m4_0,0.575000,0.570663,0.714340,0.544507,0.000,0.0
4_0,0.647414,0.572734,0.693172,0.535000,0.500,0.0
6_0,0.552857,0.590588,0.559643,0.000000,0.000,0.0
0_6,0.540000,0.000000,0.552540,0.000000,0.000,0.0
0_4,0.568889,0.000000,0.625221,0.000000,0.000,0.0
0_m4,0.000000,0.000000,0.618162,0.000000,0.537,0.0
0_m6,0.560000,0.000000,0.554519,0.000000,0.000,0.0


In [78]:
yolov3_confi

,car,bus,person,chair,suitcase,monitor
m6_0,0.902288,0.784651,0.951311,0.582500,0.649600,0.0
m4_0,0.954194,0.933354,0.948934,0.711967,0.000000,0.0
4_0,0.979711,0.877984,0.979224,0.944486,0.000000,0.0
6_0,0.986115,0.756000,0.991917,0.889182,0.000000,0.0
0_6,0.929077,0.000000,0.994538,0.919524,0.637500,0.0
0_4,0.825441,0.000000,0.970887,0.806000,0.000000,0.0
0_m4,0.928106,0.000000,0.988629,0.703049,0.595909,0.0
0_m6,0.947724,0.000000,0.995966,0.867619,0.652353,0.0
